In [ ]:
import sys
sys.path.append('../python')
from hydromap import Control
from delineate import delineate
from misc import get_precipitation, get_pet

import pandas as pd
from ipyleaflet import Map, Marker
import ipywidgets as widgets
from IPython.display import display

In [ ]:
center = (-10, -60)
zoom = 4
m = Map(center=center, zoom=zoom, interpolation='nearest')
m

In [ ]:
message = widgets.Label()
display(message)
control = Control(m, message)
widgets.interact(control.show, width=widgets.FloatSlider(min=3/1200,max=0.1,step=1/1200,value=0.05))
widgets.interact(control.show, elevation=widgets.BoundedIntText(min=0,max=10000,step=1,value=0, description='Flood elevation:', disabled=False))
m.on_interaction(control.show)

# Right-click menu
A menu pops up when you right-click on a location (in Jupyter Lab, use `shift+right-click` to get rid of Lab's own context menu). It allows you to:
- show/hide flow accumulation or elevation. Flow accumulation and elevation are only visible on a square around the mouse position, because it takes too much memory for the browser to show it all, and also because it is context dependent: the color map will fit the range of displayed values. The width of this square can be changed with the slider above (values are in degrees). Please note that the flow accumulation image might not correspond to the rivers you see on the map, especially far from the mouse position, first because they come from different sources, and second because the image is displayed as-is (EPSG:4326) and not reprojected to Web Mercator (EPSG:3857). The difference is small enough near the mouse position anyway, and the purpose of e.g. showing flow accumulation is just to check that you are on a river before delineating a watershsed.
- delineate a watershed. Delineation will start at the pixel the mouse is positioned on. You can zoom in until you make sure you are on the river of interest, and you can also check the lat/lon coordinates and flow accumulation numbers displayed above.
- delineate a floodplain. Delineation will start at the pixel the mouse is positioned on.

# Marker

Use the following cell to display a marker on the map at a particular location. You can then drag it with the mouse and get its new location back with the next cell.

In [ ]:
latlon = -0.6672326905216578, -51.08033180236817 # Amazon outlet
marker = Marker(location=latlon)
m.add_layer(marker)

In [ ]:
marker.location

In [ ]:
m.remove_layer(marker)

If you set a marker with the menu, you can get its position back:

In [ ]:
if flow.marker is not None:
    flow.marker.location

The following not only delineates the Amazon basin, but it creates a hydrologic partition, including the subbasins whose outlet has a virtual station. The whole thing takes days to compute, but the result has been stored in `pangeo-data/ws_mask/amazon` Google Cloud Storage bucket.

In [ ]:
df = pd.read_pickle('../data/amazonas/amazonas.pkl')
sub_latlon = df[['new_lat', 'new_lon']].dropna().values

lat, lon = -0.6672326905216578, -51.08033180236817 # Amazon outlet
delineate(lat, lon, sub_latlon, acc_delta=1_000_000, progress=True)

In [ ]:
d0, d1 = '2000-03-01 12:00:00', '2000-03-01 21:00:00'
precipitation = get_precipitation(d0, d1, 'tmp/ds_mask/0', None)

In [ ]:
pet = get_pet(d0, d1, 'tmp/ds_mask/0', None)

In [ ]:
df = pd.DataFrame()
df['precipitation'] = precipitation
df['pet'] = pet
df.plot(figsize=(15, 5))